In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import csv
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA,IncrementalPCA, FactorAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import validation_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve

plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib inline

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [2]:
apptrain=pd.read_csv("modedapptrain.csv")
apptest=pd.read_csv("application_test.csv")

In [3]:
apptrain.drop("Unnamed: 0", axis=1,inplace=True)

In [4]:
apptrain.drop(apptrain[apptrain["AMT_ANNUITY"].isnull()].index,inplace=True)

In [5]:
apptest.CODE_GENDER[apptest.CODE_GENDER == 'M'] = 1
apptest.CODE_GENDER[apptest.CODE_GENDER == 'F'] = 0
apptest.CODE_GENDER[apptest.CODE_GENDER == 'XNA'] = 1





apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE.isnull()] = 0
apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE == 'Unaccompanied'] = 1
apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE == 'Family'] = 2
apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE == 'Spouse, partner'] = 3
apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE == 'Children'] = 4
apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE == 'Other_A'] = 5
apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE == 'Other_B'] = 6
apptest.NAME_TYPE_SUITE[apptest.NAME_TYPE_SUITE == 'Group of people'] = 7

apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE.isnull()] = 0
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Laborers'] = 1
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Core staff'] = 2
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Accountants'] = 3
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Managers'] = 4
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Drivers'] = 5
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Sales staff'] = 6
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Cleaning staff'] = 7
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Cooking staff'] = 8
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Private service staff'] = 9
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Medicine staff'] = 10
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Security staff'] = 11
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'High skill tech staff'] = 12
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Waiters/barmen staff'] = 13
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Low-skill Laborers'] = 14
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Realty agents'] = 15
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'Secretaries'] = 16
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'IT staff'] = 17
apptest.OCCUPATION_TYPE[apptest.OCCUPATION_TYPE == 'HR staff'] = 18

D:\product\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\product\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\product\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\product\Anaconda\lib\site-packa

In [6]:
car={'Y':1,'N':0}
apptest.FLAG_OWN_CAR = [car[item] for item in apptest.FLAG_OWN_CAR]

realty={'Y':1,'N':0}
apptest.FLAG_OWN_REALTY = [realty[item] for item in apptest.FLAG_OWN_REALTY]

housing={'House / apartment' : 1,'Rented apartment' : 2,'With parents' : 3,
'Municipal apartment' : 4,'Office apartment' :5,'Co-op apartment' : 6}
apptest.NAME_HOUSING_TYPE = [housing[item] for item in apptest.NAME_HOUSING_TYPE]

income={'Working' : 1,'State servant' : 2,'Commercial associate' : 3,'Pensioner' : 4,
        'Unemployed' :5,'Student' : 6,'Businessman' : 7,'Maternity leave' : 8}
apptest.NAME_INCOME_TYPE = [income[item] for item in apptest.NAME_INCOME_TYPE]

education={'Secondary / secondary special' : 1,'Higher education' : 2,'Incomplete higher' : 3,
           'Lower secondary' : 4,'Academic degree' : 5}
apptest.NAME_EDUCATION_TYPE = [education[item] for item in apptest.NAME_EDUCATION_TYPE]

contract={'Cash loans' : 1,'Revolving loans' : 2}
apptest.NAME_CONTRACT_TYPE = [contract[item] for item in apptest.NAME_CONTRACT_TYPE]

familystatus={'Single / not married':1, 'Married':2, 'Civil marriage':3, 'Widow':4,
       'Separated':5, 'Unknown':0}
apptest.NAME_FAMILY_STATUS = [familystatus[item] for item in apptest.NAME_FAMILY_STATUS]

day={'WEDNESDAY':4, 'MONDAY':2, 'THURSDAY':5, 'SUNDAY':1, 'SATURDAY':7, 'FRIDAY':6,
       'TUESDAY':3}
apptest.WEEKDAY_APPR_PROCESS_START=[day[item] for item in apptest.WEEKDAY_APPR_PROCESS_START]

In [7]:
org=apptest.ORGANIZATION_TYPE.unique()

organization={}

num=1
for item in org:
    organization[item]=num
    num+=1
    
apptest.ORGANIZATION_TYPE=[organization[item] for item in apptest.ORGANIZATION_TYPE]

In [13]:
a=((apptest.isnull().sum()/apptest.shape[0])*100).sort_values(ascending=False)
a[a>40]

COMMONAREA_MODE                 68.716150
COMMONAREA_MEDI                 68.716150
COMMONAREA_AVG                  68.716150
NONLIVINGAPARTMENTS_AVG         68.412523
NONLIVINGAPARTMENTS_MODE        68.412523
NONLIVINGAPARTMENTS_MEDI        68.412523
FONDKAPREMONT_MODE              67.284179
LIVINGAPARTMENTS_AVG            67.249302
LIVINGAPARTMENTS_MODE           67.249302
LIVINGAPARTMENTS_MEDI           67.249302
FLOORSMIN_MEDI                  66.605121
FLOORSMIN_MODE                  66.605121
FLOORSMIN_AVG                   66.605121
OWN_CAR_AGE                     66.289184
YEARS_BUILD_MODE                65.275726
YEARS_BUILD_MEDI                65.275726
YEARS_BUILD_AVG                 65.275726
LANDAREA_AVG                    57.964057
LANDAREA_MEDI                   57.964057
LANDAREA_MODE                   57.964057
BASEMENTAREA_MEDI               56.706466
BASEMENTAREA_MODE               56.706466
BASEMENTAREA_AVG                56.706466
NONLIVINGAREA_MEDI              53

In [15]:
nullcolumnlist=[
"COMMONAREA_MODE"
,"COMMONAREA_MEDI"
,"COMMONAREA_AVG"        
,"NONLIVINGAPARTMENTS_AVG"
,"NONLIVINGAPARTMENTS_MODE"
,"NONLIVINGAPARTMENTS_MEDI"
,"FONDKAPREMONT_MODE"      
,"LIVINGAPARTMENTS_AVG"    
,"LIVINGAPARTMENTS_MODE"   
,"LIVINGAPARTMENTS_MEDI"   
,"FLOORSMIN_MEDI"          
,"FLOORSMIN_MODE"          
,"FLOORSMIN_AVG"           
,"OWN_CAR_AGE"             
,"YEARS_BUILD_MODE"        
,"YEARS_BUILD_MEDI"        
,"YEARS_BUILD_AVG"             
,"LANDAREA_AVG"                
,"LANDAREA_MEDI"               
,"LANDAREA_MODE"               
,"BASEMENTAREA_MEDI"           
,"BASEMENTAREA_MODE"           
,"BASEMENTAREA_AVG"            
,"NONLIVINGAREA_MEDI"          
,"NONLIVINGAREA_MODE"          
,"NONLIVINGAREA_AVG"           
,"ELEVATORS_MODE"              
,"ELEVATORS_MEDI"              
,"ELEVATORS_AVG"               
,"WALLSMATERIAL_MODE"          
,"APARTMENTS_MEDI"             
,"APARTMENTS_AVG"              
,"APARTMENTS_MODE"             
,"HOUSETYPE_MODE"              
,"ENTRANCES_AVG"               
,"ENTRANCES_MEDI"              
,"ENTRANCES_MODE"              
,"LIVINGAREA_AVG"              
,"LIVINGAREA_MODE"             
,"LIVINGAREA_MEDI"             
,"FLOORSMAX_AVG"               
,"FLOORSMAX_MODE"              
,"FLOORSMAX_MEDI"              
,"YEARS_BEGINEXPLUATATION_AVG" 
,"YEARS_BEGINEXPLUATATION_MODE"
,"YEARS_BEGINEXPLUATATION_MEDI"
,"TOTALAREA_MODE"              
,"EMERGENCYSTATE_MODE"         
,"EXT_SOURCE_1"                
]

In [16]:
for a in nullcolumnlist:
    try:
        apptest[a].fillna(apptest[a].mean(),inplace=True)
    except:
        print(a)



FONDKAPREMONT_MODE
WALLSMATERIAL_MODE
HOUSETYPE_MODE
EMERGENCYSTATE_MODE


In [17]:
#apptest["EMERGENCYSTATE_MODE"].unique()
#apptest["HOUSETYPE_MODE"].unique()
#apptest["FONDKAPREMONT_MODE"].unique()
#apptest["WALLSMATERIAL_MODE"]

apptest["EMERGENCYSTATE_MODE"].fillna("Missing",inplace=True)
apptest["FONDKAPREMONT_MODE"].fillna("Missing",inplace=True)
apptest["HOUSETYPE_MODE"].fillna("Missing",inplace=True)
apptest["WALLSMATERIAL_MODE"].fillna("Missing",inplace=True)

In [18]:
emergencystate={'Yes':1,'No':0,'Missing':3}
apptest.EMERGENCYSTATE_MODE = [emergencystate[item] for item in apptest.EMERGENCYSTATE_MODE]

housetype={'block of flats':1,'terraced house':0,'Missing':3,'specific housing':4}
apptest.HOUSETYPE_MODE = [housetype[item] for item in apptest.HOUSETYPE_MODE]

wallmaterial={'Stone, brick':1, 'Block':2, 'Missing':3, 'Panel':4, 'Mixed':5, 'Wooden':6, 'Others':7,
               'Monolithic':8}
apptest.WALLSMATERIAL_MODE = [wallmaterial[item] for item in apptest.WALLSMATERIAL_MODE]

fondkapre={'reg oper account':1, 'Missing':3, 'org spec account':2,
       'reg oper spec account':4, 'not specified':3}
apptest.FONDKAPREMONT_MODE=[fondkapre[item] for item in apptest.FONDKAPREMONT_MODE]

In [19]:
a=((apptest.isnull().sum()/apptest.shape[0])*100).sort_values(ascending=False)
a[a>0]

EXT_SOURCE_3                  17.782701
AMT_REQ_CREDIT_BUREAU_YEAR    12.409732
AMT_REQ_CREDIT_BUREAU_MON     12.409732
AMT_REQ_CREDIT_BUREAU_WEEK    12.409732
AMT_REQ_CREDIT_BUREAU_DAY     12.409732
AMT_REQ_CREDIT_BUREAU_HOUR    12.409732
AMT_REQ_CREDIT_BUREAU_QRT     12.409732
OBS_60_CNT_SOCIAL_CIRCLE       0.059495
DEF_60_CNT_SOCIAL_CIRCLE       0.059495
OBS_30_CNT_SOCIAL_CIRCLE       0.059495
DEF_30_CNT_SOCIAL_CIRCLE       0.059495
AMT_ANNUITY                    0.049237
EXT_SOURCE_2                   0.016412
dtype: float64

In [20]:
apptest["EXT_SOURCE_3"].fillna(apptest["EXT_SOURCE_3"].mean(),inplace=True)
apptest["EXT_SOURCE_2"].fillna(apptest["EXT_SOURCE_2"].mean(),inplace=True)

In [21]:
apptest["AMT_REQ_CREDIT_BUREAU_HOUR"].fillna(value=0,inplace=True)
apptest["AMT_REQ_CREDIT_BUREAU_DAY"].fillna(value=0,inplace=True)
apptest["AMT_REQ_CREDIT_BUREAU_QRT"].fillna(value=0,inplace=True)
apptest["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(value=0,inplace=True)
apptest["AMT_REQ_CREDIT_BUREAU_MON"].fillna(value=0,inplace=True)
apptest["AMT_REQ_CREDIT_BUREAU_WEEK"].fillna(value=0,inplace=True)

In [22]:
apptest["OBS_30_CNT_SOCIAL_CIRCLE"].fillna(0,inplace=True)
apptest["DEF_30_CNT_SOCIAL_CIRCLE"].fillna(0,inplace=True)
apptest["OBS_60_CNT_SOCIAL_CIRCLE"].fillna(0,inplace=True)
apptest["DEF_60_CNT_SOCIAL_CIRCLE"].fillna(0,inplace=True)

In [24]:
#apptest[apptest["AMT_GOODS_PRICE"].isnull()].count() #ans is zero

In [ ]:
#apptest[apptest["CNT_FAM_MEMBERS"].isnull()].count()
#apptest[apptest["DAYS_LAST_PHONE_CHANGE"].isnull()].count()
#apptest[""].fillna(0,inplace=True)
#apptest["DAYS_LAST_PHONE_CHANGE"].fillna(apptest["DAYS_LAST_PHONE_CHANGE"].min(),inplace=True)

In [27]:
a=((apptest.isnull().sum()/apptest.shape[0])*100).sort_values(ascending=False)
a[a>0]

AMT_ANNUITY    0.049237
dtype: float64

In [29]:
apptest.drop(apptest[apptest["AMT_ANNUITY"].isnull()].index,inplace=True)

In [30]:
apptest.to_csv("modedapptest.csv",sep=',',encoding='utf-8')

In [31]:
sm=SMOTE(random_state=15,ratio='minority')

In [32]:
x_res,y_res=sm.fit_sample(X=apptrain.drop("TARGET",axis=1),y=apptrain["TARGET"])

In [33]:
rfclf = RandomForestClassifier(n_estimators=30,random_state=12)

In [34]:
rfclf.fit(x_res, y_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [54]:
probs = rfclf.predict_proba(apptest)

In [63]:
dd={"SK_ID_CURR":apptest[["SK_ID_CURR"]],"TARGET":probs[:,1]}

In [92]:
probab=pd.DataFrame(probs[:,1])

In [97]:
probab.count

<bound method DataFrame.count of               0
0      0.266667
1      0.633333
2      0.366667
3      0.166667
4      0.233333
5      0.066667
6      0.433333
7      0.500000
8      0.166667
9      0.400000
10     0.233333
11     0.466667
12     0.433333
13     0.333333
14     0.433333
15     0.200000
16     0.433333
17     0.566667
18     0.233333
19     0.333333
20     0.100000
21     0.400000
22     0.200000
23     0.300000
24     0.533333
25     0.633333
26     0.266667
27     0.400000
28     0.333333
29     0.333333
...         ...
48690  0.600000
48691  0.500000
48692  0.366667
48693  0.600000
48694  0.200000
48695  0.233333
48696  0.166667
48697  0.500000
48698  0.400000
48699  0.600000
48700  0.200000
48701  0.666667
48702  0.700000
48703  0.100000
48704  0.633333
48705  0.533333
48706  0.400000
48707  0.166667
48708  0.400000
48709  0.333333
48710  0.600000
48711  0.333333
48712  0.066667
48713  0.233333
48714  0.166667
48715  0.533333
48716  0.300000
48717  0.133333
48718  

In [67]:
probs[:,1]

array([0.26666667, 0.63333333, 0.36666667, ..., 0.13333333, 0.23333333,
       0.43333333])

In [73]:
finalOutput=probab.join(apptest[["SK_ID_CURR"]])

In [74]:
finalOutput.columns=["SK_ID_CURR","TARGET"]

In [76]:
finalOutput=apptest[["SK_ID_CURR"]].join(probab)

In [77]:
finalOutput.columns=["SK_ID_CURR","TARGET"]

In [79]:
apptest.shape

(48720, 121)

In [81]:
probab.shape

(48720, 1)

In [95]:
Submission.

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,0
count,48720.000000,48720.000000,48720.000000,48720.000000,48720.000000,4.872000e+04,4.872000e+04,48720.000000,4.872000e+04,48720.000000,...,48720.0,48720.0,48720.0,48720.000000,48720.00000,48720.000000,48720.000000,48720.000000,48720.000000,48696.000000
mean,277830.123892,1.009011,0.337213,0.690640,0.397085,1.784248e+05,5.167326e+05,29426.240209,4.626091e+05,2.111412,...,0.0,0.0,0.0,0.001847,0.00158,0.002443,0.008149,0.478961,1.737274,0.344350
std,103166.090841,0.094497,0.472763,0.462234,0.709085,1.015256e+05,3.653689e+05,16016.368315,3.366722e+05,1.219528,...,0.0,0.0,0.0,0.043416,0.04319,0.050594,0.103884,0.673399,1.841082,0.171189
min,100001.000000,1.000000,0.000000,0.000000,0.000000,2.694150e+04,4.500000e+04,2295.000000,4.500000e+04,1.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,188593.250000,1.000000,0.000000,0.000000,0.000000,1.125000e+05,2.606400e+05,17973.000000,2.250000e+05,1.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.200000
50%,277582.000000,1.000000,0.000000,1.000000,0.000000,1.575000e+05,4.500000e+05,26199.000000,3.960000e+05,1.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.333333
75%,367580.250000,1.000000,1.000000,1.000000,1.000000,2.250000e+05,6.750000e+05,37390.500000,6.300000e+05,3.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,1.000000,3.000000,0.466667
max,456250.000000,2.000000,1.000000,1.000000,20.000000,4.410000e+06,2.245500e+06,180576.000000,2.245500e+06,7.000000,...,0.0,0.0,0.0,2.000000,2.00000,2.000000,6.000000,7.000000,17.000000,0.900000


In [99]:
((Submission.isnull().sum()/Submission.shape[0])*100).sort_values(ascending=False)


0                               0.049261
REGION_RATING_CLIENT_W_CITY     0.000000
HOUR_APPR_PROCESS_START         0.000000
REG_REGION_NOT_LIVE_REGION      0.000000
REG_REGION_NOT_WORK_REGION      0.000000
LIVE_REGION_NOT_WORK_REGION     0.000000
REG_CITY_NOT_LIVE_CITY          0.000000
REG_CITY_NOT_WORK_CITY          0.000000
LIVE_CITY_NOT_WORK_CITY         0.000000
ORGANIZATION_TYPE               0.000000
EXT_SOURCE_1                    0.000000
EXT_SOURCE_2                    0.000000
EXT_SOURCE_3                    0.000000
APARTMENTS_AVG                  0.000000
BASEMENTAREA_AVG                0.000000
YEARS_BEGINEXPLUATATION_AVG     0.000000
YEARS_BUILD_AVG                 0.000000
COMMONAREA_AVG                  0.000000
ELEVATORS_AVG                   0.000000
ENTRANCES_AVG                   0.000000
FLOORSMAX_AVG                   0.000000
FLOORSMIN_AVG                   0.000000
LANDAREA_AVG                    0.000000
LIVINGAPARTMENTS_AVG            0.000000
LIVINGAREA_AVG  

In [100]:
Submission[Submission[0].isnull()]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,0
48720,456011,1,0,1,1,1,171000.0,323460.0,23134.5,270000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
48721,456013,1,0,0,1,1,130500.0,468333.0,37129.5,382500.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
48722,456028,1,1,0,1,0,220500.0,166500.0,13284.0,166500.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,NaN
48723,456058,1,0,0,0,1,67500.0,315000.0,16213.5,315000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
48724,456111,1,0,0,1,1,112500.0,514710.0,17707.5,387000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
48725,456114,1,0,0,0,0,45000.0,450000.0,24543.0,450000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0,NaN
48726,456115,1,1,0,1,1,225000.0,1002339.0,37917.0,846000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
48727,456116,1,0,0,0,1,180000.0,149256.0,15291.0,135000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
48728,456119,1,0,0,1,0,247500.0,685386.0,38398.5,585000.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,NaN
48729,456120,1,0,0,0,2,81000.0,135000.0,16020.0,135000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
